In [1]:
import mediapipe as mp
from pymongo import MongoClient
import numpy as np
import pandas as pd

BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

modelPath = 'face_landmarker.task'

options = FaceLandmarkerOptions(
    base_options=BaseOptions(
        model_asset_path=modelPath
    ),
    output_facial_transformation_matrixes=True,
    running_mode=VisionRunningMode.IMAGE)

landmarker = FaceLandmarker.create_from_options(options)

W0000 00:00:1769987218.508865  313415 face_landmarker_graph.cc:180] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
I0000 00:00:1769987218.574977  313415 gl_context.cc:407] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M1 Max
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1769987218.576252  313420 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1769987218.583448  313425 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [2]:
csv = pd.concat([pd.read_csv('Data/fairface_label_val.csv'), pd.read_csv('Data/fairface_label_train.csv')])
csv = csv[(csv['gender'] == 'Female') & (csv['age'] != '3-9')]
csv = csv.drop(['race', 'service_test', 'gender'], axis=1)
csv = csv.reset_index(drop=True)

In [3]:
client = MongoClient('mongodb://localhost:55001/')
db = client['face_database']
collection = db['faces']

In [10]:
for index, row in csv.iterrows():
    mp_image = mp.Image.create_from_file('Data/fairface-img-margin125-trainval/' + row['file'])
    face_landmarker_result = landmarker.detect(mp_image)
    if len(face_landmarker_result.face_landmarks) == 0:
        continue
    landmarks = face_landmarker_result.face_landmarks[0]
    landmarksDict = [{'x': lm.x, 'y': lm.y, 'z': lm.z} for lm in landmarks]
    transformationMatrix = face_landmarker_result.facial_transformation_matrixes[0]
    inverseMatrix = np.linalg.inv(transformationMatrix)
    transformedLandmarks = [np.matmul([lm.x, lm.y, lm.z, 1], inverseMatrix) for lm in landmarks]
    transformedLandmarksDict = [{'x': lm[0], 'y': lm[1], 'z': lm[2]} for lm in transformedLandmarks]
    document = {
        'age': row['age'],
        'landmarks': landmarksDict,
        'transformedLandmarks': transformedLandmarksDict,
        'transformationMatrix': face_landmarker_result.facial_transformation_matrixes[0].tolist()
    }
    collection.insert_one(document)